# Análisis y visualización de datos con python
# 7. Data Wrangling

    - a Tablas pivote: filas a columnas
    - b Melt: columnas a filas
    - c División-aplicación-combinación
    - d Agrupar por características
    - e Funciones de agregación
    - f Unir tablas uno a uno
    - g Unir tablas muchos a uno
    
--- 

Empezaremos cargando los datos en formato pickle para asegurarnos que los tipos de datos se respeten.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import load

in_file = "data_clean/MFC_ActualizacionNov2022_clean.pkl"
with open(in_file, 'rb') as f:
     df = load(f)
df.tail()


,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_inhumación_diasemana,Fecha_inhumación_semanaaño,Fecha_defunción,Fecha_defunción_diasemana,Fecha_defunción_semanaaño,Restos_tipo,Sexo,Edad,Edad_años,Conocido_Desconocido,Primer_apellido,Segundo_Apellido,Nombres,Nombre_completo,Institución_origen,Tipo_institución_origen
ID,,,,,,,,,,,,,,,,,,,
XX-P996,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,NaT,NaN,NaN,Cadáver,Masculino,42,42.0,Conocido,Flores,Rosette,Sergio,Sergio Flores Rosette,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública
XX-P997,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2017-10-27,Viernes,43.0,Cadáver,Masculino,64,64.0,Conocido,Juarez,Chavez,Juan Carlos,Juan Carlos Juarez Chavez,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública
XX-P998,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-08-18,Sábado,33.0,Cadáver,Masculino,NaN,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial
XX-P999,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-05-22,Martes,21.0,Cadáver,Femenino,1,1.0,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial
XX-P1000,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-05-29,Martes,22.0,Cadáver,Masculino,NaN,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial


## 7.a Tablas pivote: filas a columnas
Una tabla pivote (o tabla dinámica) es una herramienta que se utiliza en hojas de cálculo y programas de análisis de datos para resumir grandes cantidades de información en un formato más fácil de entender y analizar.
En una tabla pivote, se puede tomar un conjunto de datos y resumirlos en una tabla que muestre las relaciones entre diferentes variables. La tabla pivote permite crear informes de resumen y análisis de datos dinámicos que se pueden actualizar fácilmente en función de los cambios en los datos subyacentes.

La tabla pivote se puede utilizar para realizar diferentes operaciones, como la suma, el promedio, el conteo, el máximo y el mínimo, en los datos de origen. También se pueden aplicar filtros y ordenar los datos en función de las necesidades del usuario. La tabla pivote es especialmente útil para analizar grandes cantidades de datos y para visualizar tendencias y patrones.

La función `.pivot_table()` se utiliza para crear tablas pivotes en un DataFrame, y permite agregar y resumir datos según uno o varios campos (o columnas) de interés. Esta función toma los siguientes parámetros:

* values: es la columna o columna(s) que se quieren resumir
* index: es la columna(s) que se utilizan como índice (filas) en la nueva tabla
* columns: es la columna(s) que se utilizan como columnas en la nueva tabla
* aggfunc: es la función de agregación que se desea aplicar a los valores en la tabla pivote. Por defecto es la función de agregación mean (promedio).
* fill_value: es el valor que se utiliza para rellenar los valores faltantes (NaN) en la tabla pivote.
* dropna: ignorar o no los valores faltantes

Por ejemplo, calculemos el promedio de las edades para el tipos de restos y el sexo

**Nota**: Si no especificamos el parametro `values` se aplicará la funcion de agregación `aggfunc` a todas las columnas de la tabla. 

In [2]:
pd.pivot_table(df, index='Restos_tipo', columns='Sexo', values='Edad_años', aggfunc='mean')

Sexo,Femenino,Masculino
Restos_tipo,,
Cadáver,61.027778,54.836449
Restos humanos,19.200000,31.000000
Restos cremados,68.140351,68.376712
Miembros,55.666667,36.537037



Si se quiere calcular una tabla pivote para una variable derivada de los datos es necesario generarla y agregarla a la tabla. Por ejemplo, supongamos que queremos calcular el número de inhumanciones por año y tipo de restos. En ese caso es necesario usar generar una nueva variable "Fecha_inhumacion_año".

In [3]:
df['Fecha_inhumacion_año'] = df['Fecha_inhumación'].dt.year
df['Fecha_defunción_año'] = df['Fecha_defunción'].dt.year

Ahora vamos calcular el tamaño de cada grupo con `size`.

**Nota**: La función  `count` es muy similar pero se aplica a cada columna no cuenta los `nan` , por lo que los valores pueden variar.

In [4]:
pd.pivot_table(df, index='Restos_tipo', columns='Fecha_inhumacion_año',
               aggfunc='size', dropna=False)

Fecha_inhumacion_año,2017,2018,2019,2020
Restos_tipo,,,,
Cadáver,160,191,294,1
Restos humanos,11,4,5,0
Restos cremados,0,100,117,0
Restos óseos,1,0,0,0
Feto,2,16,22,0
Miembros,28,24,24,0


Podemos usar varias columnas cualquiera de los ejes de la tabla pivote.

In [5]:
pd.pivot_table(df, index=['Sexo','Restos_tipo'],
               columns=['Fecha_inhumacion_año'], 
               aggfunc='size')

Fecha_inhumacion_año           2017  2018  2019  2020
Sexo          Restos_tipo                            
Femenino      Cadáver            20    29    30     1
              Restos humanos      2     0     4     0
              Restos cremados     0    25    35     0
              Restos óseos        0     0     0     0
              Feto                0     7     1     0
              Miembros            4     4     7     0
Indeterminado Cadáver             0     0     1     0
              Restos humanos      0     1     1     0
              Restos cremados     0     0     0     0
              Restos óseos        1     0     0     0
              Feto                1     1     4     0
              Miembros            1     1     1     0
Masculino     Cadáver           140   162   263     0
              Restos humanos      9     3     0     0
              Restos cremados     0    75    82     0
              Restos óseos        0     0     0     0
              Feto                1     8    17     0
              Miembros           23    19    16     0

Es posible utilizar otras funciones, por ejemplo usemos `pd.Series.mode` para regresar el valor más común.

In [6]:
pd.pivot_table(df, values='Tipo_institución_origen', 
               index='Restos_tipo', columns='Fecha_inhumacion_año', 
               aggfunc=pd.Series.mode, dropna=False)

Fecha_inhumacion_año,2017,2018,2019,2020
Restos_tipo,,,,
Cadáver,Institución judicial,Institución judicial,Institución judicial,Institución judicial
Restos humanos,Institución judicial,Institución judicial,Institución judicial,NaN
Restos cremados,NaN,Escuela pública,Escuela pública,NaN
Restos óseos,Institución judicial,NaN,NaN,NaN
Feto,Institución judicial,Institución judicial,Institución judicial,NaN
Miembros,Institución judicial,Institución judicial,Institución judicial,NaN


También se pueden definir funciones propias para procesar los datos.

In [7]:
def unir_textos_unicos_en_serie(series):
    textos = series.sort_values().unique()
    textos = ', '.join(textos)
    return textos

pd.pivot_table(df, values='Fecha_inhumación_diasemana', 
               index='Restos_tipo', columns='Fecha_inhumacion_año', 
               aggfunc=unir_textos_unicos_en_serie, 
               dropna=False)

Fecha_inhumacion_año,2017,2018,2019,2020
Restos_tipo,,,,
Cadáver,"Jueves, Viernes, Sábado","Martes, Miercoles, Jueves, Viernes, Sábado, Do...","Lunes, Martes, Miercoles, Jueves, Sábado",Lunes
Restos humanos,Sábado,Sábado,"Miercoles, Sábado",NaN
Restos cremados,NaN,Sábado,"Miercoles, Sábado",NaN
Restos óseos,Miercoles,NaN,NaN,NaN
Feto,Sábado,Sábado,Sábado,NaN
Miembros,"Martes, Miercoles, Viernes, Sábado",Sábado,Sábado,NaN


## 7.b Melt: columnas a filas

La función `.melt()` sirve para transformar y reorganizar un conjunto de datos en formato "ancho" en un formato "largo". La transformación implica tomar columnas de datos y reorganizarlas en filas.
La función permite especificar qué columnas de un DataFrame deben mantenerse como identificadores y cuáles deben transformarse en variables. También permite renombrar las columnas resultantes para facilitar el análisis de los datos.

Por ejemplo, veamos la siguiente tabla pivote.

In [8]:
df_edad_promedio = pd.pivot_table(df, values='Edad_años', 
                                  index='Restos_tipo', columns='Sexo')
df_edad_promedio

Sexo,Femenino,Masculino
Restos_tipo,,
Cadáver,61.027778,54.836449
Restos humanos,19.200000,31.000000
Restos cremados,68.140351,68.376712
Miembros,55.666667,36.537037


Supongamos que queremos transformarla a formato de tidy data, en ese caso cada observación debería de estar en una fila.

El primer paso es volver el indice en una columna de valores.

In [9]:
df_edad_promedio = df_edad_promedio.reset_index()
df_edad_promedio

Sexo,Restos_tipo,Femenino,Masculino
0,Cadáver,61.027778,54.836449
1,Restos humanos,19.200000,31.000000
2,Restos cremados,68.140351,68.376712
3,Miembros,55.666667,36.537037


La función `.melt()` transforma los datos de las columnas a filas.

En cierta forma esta función el la inversa de `.pivot_table()`, aunque se puede perder información, como vemos en este caso.

In [10]:
df_edad_promedio = pd.melt(df_edad_promedio, value_name='Edad_promedio',
                           id_vars='Restos_tipo', value_vars=['Femenino','Masculino'],  )
df_edad_promedio

,Restos_tipo,Sexo,Edad_promedio
0,Cadáver,Femenino,61.027778
1,Restos humanos,Femenino,19.200000
2,Restos cremados,Femenino,68.140351
3,Miembros,Femenino,55.666667
4,Cadáver,Masculino,54.836449
5,Restos humanos,Masculino,31.000000
6,Restos cremados,Masculino,68.376712
7,Miembros,Masculino,36.537037


## 7.c División-aplicación-combinación

El pipeline [split-apply-combine](https://pandas.pydata.org/docs/user_guide/groupby.html) es un enfoque muy común en el análisis de datos y se refiere a un proceso de tres pasos:
* Split (división): El conjunto de datos original se divide en grupos basados en una o más columnas. En primer lugar, se divide el conjunto de datos en grupos basados en una o varias columnas utilizando la función groupby(). Esto genera un objeto GroupBy que contiene los grupos y se puede aplicar una función o transformación a cada grupo por separado.
* Apply (aplicación): Se aplica alguna función o transformación a cada grupo por separado. Luego, se aplica alguna función o transformación a cada grupo por separado. Esta función puede ser de varios tipos:
    * Aggregation (agregación): calcular una estadística resumen (o varias estadísticas) para cada grupo. Por ejemplo calcular la suma o tamaño de los grupos.
    * Transform (transformación): realizar algunos cálculos específicos del grupo y devolver un objeto con el mismo índice.  Por ejemplo rellenar los valores perdidos (NA) dentro de un grupo con un valor derivado de ese mismo grupo.
    * Filter (filtración): descartar algunos grupos, según un cálculo por grupo que evalúa Verdadero o Falso. Por ejemplo descartar datos que pertenecen a grupos con muy pocos miembros.
    * Alguna combinación de lo anterior
* Combine (combinación): Los resultados de cada grupo se combinan en un único DataFrame utilizando la función concat(), merge() u otra función de combinación de Pandas.

A combinación veremos una introducción a este pipeline, aunque se recomienda ver la documentación de [split-apply-combine](https://pandas.pydata.org/docs/user_guide/groupby.html).

## 7.d Agrupar por características

La función `groupby()` se utiliza para agrupar datos en función de una o varias columnas y aplicar una función de agregación a cada grupo.
Esta función divide la tabla o dataframe en subtablas o grupos de acuerdo a los valores de una o más columnas. El resultado es un objeto similar a un diccionario, donde las llaves son los valores por los que se dividió la tabla y los valores las subtablas resultantes. Este objeto es iterable y se le pueden aplicar funciones por separado a cada grupo.

In [11]:
groups = df.groupby('Sexo')

for key, data in groups:
    print('Grupo:', key)
    print('\tTipo', type(data))
    print('\tTamaño', data.shape)

Grupo: Femenino
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (169, 21)
Grupo: Indeterminado
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (13, 21)
Grupo: Masculino
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (818, 21)


Usando varias columnas para agrupar se genera un `multiindex`.

In [12]:
groups = df.groupby(['Sexo','Restos_tipo'])

for key, data in groups:
    print('Grupo:', key)
    print('\tTipo', type(data))
    print('\tTamaño', data.shape)

Grupo: ('Femenino', 'Cadáver')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (80, 21)
Grupo: ('Femenino', 'Restos humanos')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (6, 21)
Grupo: ('Femenino', 'Restos cremados')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (60, 21)
Grupo: ('Femenino', 'Feto')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (8, 21)
Grupo: ('Femenino', 'Miembros')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (15, 21)
Grupo: ('Indeterminado', 'Cadáver')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (1, 21)
Grupo: ('Indeterminado', 'Restos humanos')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (2, 21)
Grupo: ('Indeterminado', 'Restos óseos')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (1, 21)
Grupo: ('Indeterminado', 'Feto')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (6, 21)
Grupo: ('Indeterminado', 'Miembros')
	Tipo <class 'pandas.core.frame.DataFrame'>
	Tamaño (3, 21)
Grupo: ('Masculino', 'Cadáver')
	Tipo <

## 7.e Funciones de agregación

Las funciones de agregación son funciones en Pandas que se utilizan en conjunto con la función groupby() para realizar cálculos resumidos sobre un conjunto de datos agrupados. Estas funciones toman un conjunto de valores y los resumen en un solo valor. 

Algunas de las funciones de agregación más comunes son:
* mean(): promedio
* sum(): suma de valores
* size(): tamaño del grupo
* count(): conteo del grupo
* std(): desviación estandar
* var(): varianza
* sem(): error estandar de la media de los grupos
* describe(): estadísticas descriptivas
* first(): primer valor
* last(): último valor
* nth(): n-esimo valor
* min(): valor mímino
* max(): valor máximo


In [13]:
df.groupby(by=['Sexo','Restos_tipo']).size()

Sexo           Restos_tipo    
Femenino       Cadáver             80
               Restos humanos       6
               Restos cremados     60
               Restos óseos         0
               Feto                 8
               Miembros            15
Indeterminado  Cadáver              1
               Restos humanos       2
               Restos cremados      0
               Restos óseos         1
               Feto                 6
               Miembros             3
Masculino      Cadáver            565
               Restos humanos      12
               Restos cremados    157
               Restos óseos         0
               Feto                26
               Miembros            58
dtype: int64


Por ejemplo, agrupemos los datos por sexo y tipo de restos y calculemos el promedio de edad. 

**Nota**: es buena practica solo seleccionar las columnas sobre las que se va a realizar la operación de agregación

In [14]:
df[['Sexo','Restos_tipo','Edad_años']].groupby(by=['Sexo','Restos_tipo']).mean()

Edad_años
Sexo          Restos_tipo               
Femenino      Cadáver          61.027778
              Restos humanos   19.200000
              Restos cremados  68.140351
              Restos óseos           NaN
              Feto                   NaN
              Miembros         55.666667
Indeterminado Cadáver                NaN
              Restos humanos         NaN
              Restos cremados        NaN
              Restos óseos           NaN
              Feto                   NaN
              Miembros               NaN
Masculino     Cadáver          54.836449
              Restos humanos   31.000000
              Restos cremados  68.376712
              Restos óseos           NaN
              Feto                   NaN
              Miembros         36.537037

La función `aggregate()` o `agg()`se utiliza para aplicar una o varias funciones de agregación a un DataFrame. Esta función es muy útil cuando queremos aplicar diferentes funciones de agregación a diferentes columnas de un DataFrame o cuando queremos aplicar funciones de agregación personalizadas.

Por ejemplo, calculemos varias estadísticas de la edad para cada tipo de restos.

Si queremos podemos renombrar las columnas resultantes con el comando `.rename()`.

In [15]:
df[['Restos_tipo','Edad_años']].groupby(by='Restos_tipo')['Edad_años'] \
                        .agg(['min', 'max', pd.Series.mode, 'mean', 'std']) \
                        .rename(columns={ 
                                         "min": "edad minima", "max": "edad maxima", 
                                         "mode": "moda edad", "mean": "promedio edad",
                                         "std": "desviación estandar",
                                         })

,edad minima,edad maxima,moda edad,promedio edad,desviación estandar
Restos_tipo,,,,,
Cadáver,1.0,97.0,40.0,55.728000,19.111752
Restos humanos,15.0,50.0,"[22.0, 23.0, 45.0]",27.529412,11.341945
Restos cremados,20.0,96.0,78.0,68.310345,14.289345
Restos óseos,NaN,NaN,[],NaN,NaN
Feto,NaN,NaN,[],NaN,NaN
Miembros,5.0,95.0,31.0,40.695652,21.668834


También se pueden aplicar distintas funciones a cada columna. Para lograr esto se pasa a la función `agg()` un diccionario de nombres de columnas y funciones.

In [16]:
df.groupby(by=['Fecha_inhumacion_año','Fecha_inhumación_semanaaño']).agg({
   'Conocido_Desconocido':'size',
   'Restos_tipo':pd.Series.mode, 
   'Tipo_institución_origen':pd.Series.mode,
   })

Conocido_Desconocido  \
Fecha_inhumacion_año Fecha_inhumación_semanaaño                         
2017                 30                                             1   
                     40                                            32   
                     41                                            36   
                     42                                            25   
                     43                                             2   
                     44                                             6   
                     45                                            30   
                     47                                            30   
                     49                                            19   
                     50                                            21   
2018                 27                                           100   
                     28                                            23   
                     29                                            11   
                     35                                            12   
                     38                                            11   
                     39                                            21   
                     41                                            13   
                     42                                            23   
                     43                                            26   
                     46                                            10   
                     47                                            29   
                     48                                            13   
                     49                                            40   
                     50                                             3   
2019                 1                                             19   
                     2                                             23   
                     3                                             21   
                     4                                             26   
                     5                                             24   
                     6                                              4   
                     7                                             15   
                     8                                              1   
                     9                                             29   
                     11                                            24   
                     13                                           142   
                     14                                            18   
                     15                                            20   
                     16                                             4   
                     18                                            37   
                     19                                             4   
                     20                                            20   
                     21                                            23   
                     22                                             8   
2020                 15                                             1   

                                                                                       Restos_tipo  \
Fecha_inhumacion_año Fecha_inhumación_semanaaño                                                      
2017                 30                                                                    Cadáver   
                     40                                                                    Cadáver   
                     41                                                                    Cadáver   
                     42                                                                    Cadáver   
                     43                          ['Restos óseos', 'Miembros']


Las funciones de agregación también se pueden utilizar en conjunto con la función `apply()` para aplicar funciones personalizadas a los datos agrupados.

Por ejemlo, recordemos la función `unir_textos_unicos_en_serie` que habiamos definido y apliquemosla a varias columnas

In [17]:
def unir_textos_unicos_en_serie(series):
    textos = series.unique().sort_values()
    textos = ', '.join(textos)
    return textos

df[['Restos_tipo','Fecha_inhumacion_año']] \
  .groupby(by='Fecha_inhumacion_año') \
  .agg(unir_textos_unicos_en_serie) 

,Restos_tipo
Fecha_inhumacion_año,
2017,"Cadáver, Restos humanos, Restos óseos, Feto, M..."
2018,"Cadáver, Restos humanos, Restos cremados, Feto..."
2019,"Cadáver, Restos humanos, Restos cremados, Feto..."
2020,Cadáver


## 7.f Unir tablas uno a uno

Es posible unir tablas de distintas maneras usando pandas, en este tutorial veremos solo las más sencillas, pero les recomendamos ver la guia de [Merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html).

Una forma de unir dos tablas es con la función `concat()`. Esta función toma una lista de dataframes y las une por el índice o las columnas. En este caso es una unión uno a uno, es decir, cada valor del indice o columna es único y no se repiten.

Como ejemplo usaremos los siguientes dataframes:

In [18]:
df1 = pd.DataFrame( {
         "A": ["A0", "A1", "A2", "A3"],
         "B": ["B0", "B1", "B2", "B3"],
         "C": ["C0", "C1", "C2", "C3"],
         "D": ["D0", "D1", "D2", "D3"],
         },
         index=[0, 1, 2, 3],
)
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [19]:
df2 = pd.DataFrame({
         "B": ["B2", "B3", "B6", "B7"],
         "D": ["D2", "D3", "D6", "D7"],
         "F": ["F2", "F3", "F6", "F7"],
         },
         index=[2, 3, 6, 7],
      )
df2

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


Si revisamos `df1` y `df2` tienen en común los indices `[2,3]` en las filas y `['B','D']` en las columnas. Los valores `['B2','B3','D2','D3']` se encuentran en ambas tablas.

En primer lugar uniremos usando `pd.concat(axis=0)`, es decir, los valores se uniran en las columnas. El resultado es una tabla de 8 filas por 5 columnas, donde los valores de las columnas son únicas.

Nota como los indices de las filas se mantuvieron como estaban en las tablas originales y se repiten. Los indides `[2,3]` aparecen dos veces.

Esta operación puede ser vista como apilar las tablas a lo alto (filas), uniendo a lo ancho (columnas).

In [20]:
pd.concat([df1,df2], axis=0)

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


Por ejemplo, supongamos que queremos unir las dos columnas de apellidos en una sola para ver cuál es el apellido más común. Entonces usaremos `pd.concat(axis=0)`. Cada columna tiene mil filas, entonces el resultado es una columna con dos mil filas.

In [21]:
pd.concat( [df['Primer_apellido'], df['Segundo_Apellido']], axis=0 )

ID
XX-P001            
XX-P002            
XX-P003        Cruz
XX-P004            
XX-P005            
             ...   
XX-P996     Rosette
XX-P997      Chavez
XX-P998            
XX-P999            
XX-P1000           
Length: 2000, dtype: object

En segundo lugar uniremos usando `pd.concat(axis=1)`, es decir, los valores se uniran en las filas. El resultado es una tabla de 6 filas por 7 columnas, donde los valores de las filas son únicas.

Nota como los nombres de las columnas se mantuvieron como estaban en las tablas originales y se repiten. Los nombres `['B','D']` aparecen dos veces.

Esta operación puede ser vista como pegar las tablas a lo ancho (columnas), uniendo a lo alto (filas).

In [22]:
pd.concat([df1,df2], axis=1)

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


Esto puede ser usado para unir tablas con variables nuevas. 

Por ejemplo, supongamos que queremos obtener en una sola tabla las defunciones e inhumaciones por sexo y año. Una forma de hacer esto es calcular las tablas por separado y unirlas.

**Nota**: esta no es la única manera.

In [23]:
sexo_defuncion = df.pivot_table(index='Fecha_defunción_año',columns='Sexo', aggfunc='size') \
                   .rename(columns={'Femenino':'Defunciones (Femenino)',
                                    'Indeterminado':'Defunciones (Indeterminado)',
                                    'Masculino':'Defunciones (Masculino)',
                           })
sexo_defuncion

Sexo,Defunciones (Femenino),Defunciones (Indeterminado),Defunciones (Masculino)
Fecha_defunción_año,,,
2001.0,0,0,1
2012.0,1,0,0
2014.0,2,0,7
2015.0,15,0,27
2016.0,33,1,93
2017.0,31,1,202
2018.0,51,2,247
2019.0,15,3,155


In [24]:
sexo_inhumacion = df.pivot_table(index='Fecha_inhumacion_año',columns='Sexo', aggfunc='size') \
                   .rename(columns={'Femenino':'Inhumaciones (Femenino)',
                                    'Indeterminado':'Inhumaciones (Indeterminado)',
                                    'Masculino':'Inhumaciones (Masculino)',
                           })
sexo_inhumacion

Sexo,Inhumaciones (Femenino),Inhumaciones (Indeterminado),Inhumaciones (Masculino)
Fecha_inhumacion_año,,,
2017,26,3,173
2018,65,3,267
2019,77,7,378
2020,1,0,0


Ahora unimos las dos tablas por año, es decir por las filas con `axis=1`.
Además. llenaremos los valores faltantes con cero.

In [25]:
pd.concat([sexo_defuncion,sexo_inhumacion], axis=1).fillna(0)

Sexo,Defunciones (Femenino),Defunciones (Indeterminado),Defunciones (Masculino),Inhumaciones (Femenino),Inhumaciones (Indeterminado),Inhumaciones (Masculino)
2001.0,0.0,0.0,1.0,0.0,0.0,0.0
2012.0,1.0,0.0,0.0,0.0,0.0,0.0
2014.0,2.0,0.0,7.0,0.0,0.0,0.0
2015.0,15.0,0.0,27.0,0.0,0.0,0.0
2016.0,33.0,1.0,93.0,0.0,0.0,0.0
2017.0,31.0,1.0,202.0,26.0,3.0,173.0
2018.0,51.0,2.0,247.0,65.0,3.0,267.0
2019.0,15.0,3.0,155.0,77.0,7.0,378.0
2020.0,0.0,0.0,0.0,1.0,0.0,0.0


## 7.g Unir tablas muchos a uno

Otro caso de unir tablas es cuando tenemos muchos valores que se uniran a uno solo.

Por ejemplo, supongamos que tenemos la siguiente tabla con información de la institutición de origen. Esta tabla nos da información adicional que nos gustaría unir a la informacíon contenida en el MFC. 

In [26]:
df_inst = pd.read_csv('data_raw/MFC_InstitucionesOrigen.csv')
df_inst

,Nombre,Institución,Dependencia,Estado,Tipo de institución
0,Universidad Westhill - Facultad De Medicina,Universidad Westhill,Facultad De Medicina,CdMx,Escuela privada
1,Instituto De Ciencias Forenses - Tribunal Supe...,Tribunal Superior De Justicia De La,Instituto De Ciencias Forenses,CdMx,Institución judicial
2,Universidad Tominaga Nakamoto S.C. Escuela De ...,Universidad Tominaga Nakamoto S.C.,Escuela De Medicina Ciencias Basicas,CdMx,Escuela privada
3,Instituto Politecnico Nacional - Escuela Nacio...,Instituto Politecnico Nacional,Escuela Nacional De Medicina Y,CdMx,Escuela pública
4,Centro Cultural Universitario Justo Sierra,Centro Cultural Universitario Justo Sierra,NaN,CdMx,Escuela privada
5,Universidad Nacional Autonoma De Mexico - Facu...,Universidad Nacional Autonoma De Mexico,Facultad De Medicina,CdMx,Escuela pública
6,Universidad Anahuac - Facultad De Ciencias De ...,Universidad Anahuac,Facultad De Ciencias De La Salud,CdMx,Escuela privada
7,Institucion De Asistencia Privada - Escuela Li...,Institucion De Asistencia Privada,Escuela,CdMx,Escuela privada
8,Secretaria De La Defensa Nacional - Escuela Mi...,Secretaria De La Defensa Nacional,Escuela Militar De Medicina,CdMx,Escuela privada
9,The American British Cowdray Medical Center La.P.,The American British Cowdray Medical Center La.P.,NaN,CdMx,Escuela privada


En este caso, cada vez que aparece en el `df` del MFC el nombre de una `Institución_origen` nos gustaría que en la misma fila aparezcan los datos de la institución, dependencia, estado y tipo.

Como en la tabla original del MFC aparecen varias veces el nombre de una misma institución, pero en `df_inst` solo aparece una vez decimos que es una unión de muchos a uno.

En este caso uniremos a la izquierda MFC `df` en la columna `'Institución_origen'` con la tabla de instituciones `df_inst` en la columna `Nombre` a la derecha'Institución_origen'

Podemos ver como se agregaron las columnas correspondientes a la derecha de la tabla.

**Nota**: En todos los casos de unión es fundamental revisar el número de filas y columnas resultantes para garantizar que no se perdió información.

In [27]:
df

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_inhumación_diasemana,Fecha_inhumación_semanaaño,Fecha_defunción,Fecha_defunción_diasemana,Fecha_defunción_semanaaño,Restos_tipo,Sexo,...,Edad_años,Conocido_Desconocido,Primer_apellido,Segundo_Apellido,Nombres,Nombre_completo,Institución_origen,Tipo_institución_origen,Fecha_inhumacion_año,Fecha_defunción_año
ID,,,,,,,,,,,,,,,,,,,,,
XX-P001,Panteón Civil de Dolores,Inhumación,2019-01-30,Miercoles,5,2018-03-04,Domingo,9.0,Restos cremados,Masculino,...,55.0,Desconocido,,,,,Universidad Westhill - Facultad De Medicina,Escuela privada,2019,2018.0
XX-P002,Panteón Civil de Dolores,Inhumación,2019-01-26,Sábado,4,2018-12-12,Miercoles,50.0,Cadáver,Masculino,...,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2019,2018.0
XX-P003,Panteón Civil de Dolores,Inhumación,2019-01-26,Sábado,4,2018-12-13,Jueves,50.0,Cadáver,Masculino,...,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2019,2018.0
XX-P004,Panteón Civil de Dolores,Inhumación,2019-01-26,Sábado,4,2018-12-14,Viernes,50.0,Cadáver,Masculino,...,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2019,2018.0
XX-P005,Panteón Civil de Dolores,Inhumación,2019-01-26,Sábado,4,2018-12-18,Martes,51.0,Cadáver,Masculino,...,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2019,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX-P996,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,NaT,NaN,NaN,Cadáver,Masculino,...,42.0,Conocido,Flores,Rosette,Sergio,Sergio Flores Rosette,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública,2018,NaN
XX-P997,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2017-10-27,Viernes,43.0,Cadáver,Masculino,...,64.0,Conocido,Juarez,Chavez,Juan Carlos,Juan Carlos Juarez Chavez,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública,2018,2017.0
XX-P998,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-08-18,Sábado,33.0,Cadáver,Masculino,...,NaN,Desconocido,,,,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2018,2018.0


In [28]:
df.merge(df_inst, how='left', left_on='Institución_origen', right_on='Nombre')

,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_inhumación_diasemana,Fecha_inhumación_semanaaño,Fecha_defunción,Fecha_defunción_diasemana,Fecha_defunción_semanaaño,Restos_tipo,Sexo,...,Nombre_completo,Institución_origen,Tipo_institución_origen,Fecha_inhumacion_año,Fecha_defunción_año,Nombre,Institución,Dependencia,Estado,Tipo de institución
0,Panteón Civil de Dolores,Inhumación,2019-01-30,Miercoles,5,2018-03-04,Domingo,9.0,Restos cremados,Masculino,...,,Universidad Westhill - Facultad De Medicina,Escuela privada,2019,2018.0,Universidad Westhill - Facultad De Medicina,Universidad Westhill,Facultad De Medicina,CdMx,Escuela privada
1,Panteón Civil de Dolores,Inhumación,2019-01-26,Sábado,4,2018-12-12,Miercoles,50.0,Cadáver,Masculino,...,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2019,2018.0,Instituto De Ciencias Forenses - Tribunal Supe...,Tribunal Superior De Justicia De La,Instituto De Ciencias Forenses,CdMx,Institución judicial
2,Panteón Civil de Dolores,Inhumación,2019-01-26,Sábado,4,2018-12-13,Jueves,50.0,Cadáver,Masculino,...,Alberto Cruz Lucero,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2019,2018.0,Instituto De Ciencias Forenses - Tribunal Supe...,Tribunal Superior De Justicia De La,Instituto De Ciencias Forenses,CdMx,Institución judicial
3,Panteón Civil de Dolores,Inhumación,2019-01-26,Sábado,4,2018-12-14,Viernes,50.0,Cadáver,Masculino,...,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2019,2018.0,Instituto De Ciencias Forenses - Tribunal Supe...,Tribunal Superior De Justicia De La,Instituto De Ciencias Forenses,CdMx,Institución judicial
4,Panteón Civil de Dolores,Inhumación,2019-01-26,Sábado,4,2018-12-18,Martes,51.0,Cadáver,Masculino,...,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2019,2018.0,Instituto De Ciencias Forenses - Tribunal Supe...,Tribunal Superior De Justicia De La,Instituto De Ciencias Forenses,CdMx,Institución judicial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,NaT,NaN,NaN,Cadáver,Masculino,...,Sergio Flores Rosette,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública,2018,NaN,Universidad Nacional Autonoma De Mexico - Facu...,Universidad Nacional Autonoma De Mexico,Facultad De Medicina,CdMx,Escuela pública
996,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2017-10-27,Viernes,43.0,Cadáver,Masculino,...,Juan Carlos Juarez Chavez,Universidad Nacional Autonoma De Mexico - Facu...,Escuela pública,2018,2017.0,Universidad Nacional Autonoma De Mexico - Facu...,Universidad Nacional Autonoma De Mexico,Facultad De Medicina,CdMx,Escuela pública
997,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-08-18,Sábado,33.0,Cadáver,Masculino,...,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2018,2018.0,Instituto De Ciencias Forenses - Tribunal Supe...,Tribunal Superior De Justicia De La,Instituto De Ciencias Forenses,CdMx,Institución judicial
998,Panteón Civil de Dolores,Inhumación,2018-09-01,Sábado,35,2018-05-22,Martes,21.0,Cadáver,Femenino,...,,Instituto De Ciencias Forenses - Tribunal Supe...,Institución judicial,2018,2018.0,Instituto De Ciencias Forenses - Tribunal Supe...,Tribunal Superior De Justicia De La,Instituto De Ciencias Forenses,CdMx,Institución judicial




Existen muchas formas de unir tablas usando llaves, una vez mas recomendamos ver el tutorial de [merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html).

**Gracias por su atención**